In [1]:
from moviepy.audio.fx.all import *
from moviepy.editor import *
from moviepy.video.fx.all import *
from moviepy.video.tools.subtitles import SubtitlesClip

from skimage.filters import gaussian

In [2]:
video_file_path = '/home/rohit/Videos/sasha_clips/gi_e1.mp4' 

In [3]:
t_sub_start = (0,3,20.000)
t_sub_end = (0,4,53.500)

In [4]:
main_clip_original = VideoFileClip(video_file_path).subclip(t_sub_start, t_sub_end)

In [5]:
w, h = main_clip_original.size

In [6]:
main_clip_crop = crop(main_clip_original, width=h, height=h, x_center=0.5*w, y_center=0.5*h)
#main_clip_crop = crop(main_clip_original, width=607, height=1080, x_center=0.5*w, y_center=0.5*h)

In [7]:
#main_clip_crop.ipython_display(width=640, maxduration=180)
#main_clip_crop.ipython_display(t=0)
#main_clip_crop.write_videofile('/home/rohit/Videos/sasha_clips/test.mp4', logger=None)

### Adding logo and insta handle

In [8]:
##Adding logo and instahandle to top right corner
#logo = (ImageClip("/home/rohit/Videos/sasha_clips/logos/logo_symbol_only/symbol1.png")
#       .set_duration(main_clip_original.duration)
#       .set_position((640, -40))
#       .resize(0.25)
#       .crossfadeout(0.3))
#
#handle = (TextClip('SASHA COBRA', font='Montserrat-SemiBold', fontsize=25, color='white', 
#                   align='center', kerning=5)
#          .set_position((800, 52))
#          .set_duration(main_clip_original.duration)
#          .crossfadeout(0.3))

In [9]:
#Logo and handle on top left corner
logo = (ImageClip("/home/rohit/Videos/sasha_clips/logos/logo_symbol_only/symbol6.png")
       .set_duration(main_clip_crop.duration)
       .set_position((-35, -40))
       .resize(0.25)
       .crossfadeout(0.3))

handle = (TextClip('SASHA COBRA', font='Montserrat-SemiBold', fontsize=25, color='#423634', 
                   align='center', kerning=5)
          .set_position((125, 52))
          .set_duration(main_clip_crop.duration)
          .crossfadeout(0.3))

In [10]:
main_clip_logo = CompositeVideoClip([main_clip_crop, logo, handle])

In [11]:
#main_clip_logo.ipython_display(width=640, t=0)
#main_clip_logo.save_frame('/home/rohit/Downloads/logo_text_montserrat_medium_caps.png')

In [12]:
#logo = (ImageClip("/home/rohit/Videos/sasha_clips/logos/logo_name/logo_name_9.png")
#       .set_duration(main_clip_crop.duration)
#       .set_position((-30, -40))
#       .resize(0.25)
#       .crossfadeout(1.5))
#
#main_clip_logo = CompositeVideoClip([main_clip_crop, logo])

### Adding subtitles the manual way

In [13]:
#Defining file path
subtitle_file = '/home/rohit/Videos/sasha_clips/09_love_programming.srt'

In [14]:
#Defining a function that can take a string of srt timestamp
#and convert it to format that MoviePy needs
def timestamp_convert(srt_time):
    
    '''
    Expects a string in the form 'hours:min:seconds,milliseconds'
    Returns (hours, minutes, seconds.milliseconds)
    '''
    
    #Splitting on the :
    srt_time = srt_time.split(':')
    
    #Extracting hour, minute and seconds
    hour = int(srt_time[0])
    minute = int(srt_time[1])
    seconds = float(srt_time[2].replace(',', '.'))
    
    #Converting to tuple
    moviepy_time = (hour, minute, seconds)
    
    return moviepy_time

In [15]:
#Extracting contents of srt file into a list
with open(subtitle_file) as f:
    sub_content = f.readlines()
    
#print(sub_content)

#Defining a counter to keep track of the max number of 
#subtitle sections
max_subtitle_section = 0

#Defining a dictionary to keep track of subtitle content
#This is a dicionary that stores another dictionary
subtitle = {}

#Remove the \n at the end of each line
sub_content = [line.rstrip() for line in sub_content]
    
for i in range(len(sub_content)):
    
    #Check if string is number
    if sub_content[i].isdigit():
        
        #Create a temp dictionary
        temp = {}
        
        #Extract the timestamps
        timestamps = sub_content[i+1]
        
        #Split the timestamps by space
        timestamps = timestamps.split(' ')
        
        #Extracting the start and end time for the subtitle
        temp['start'] = timestamp_convert(timestamps[0])
        temp['end'] = timestamp_convert(timestamps[2])
        
        #Extract the subtitle text
        temp['text'] = sub_content[i+2]
        
        #Adding the temp dictionary to the subtitle dictionary
        subtitle[int(sub_content[i])] = temp
        

In [16]:
#Defining list for concatenation
clips_for_concatenation = []

#Looping through each subtitle section
for i in subtitle.keys():
    
    #Extracting the clip that is in between two subtitles
    #and appending to the list for concatenation
    if i != len(subtitle.keys()):
        
        #Extracting the clip
        subtitle[i]['clip'] = main_clip_logo.subclip(subtitle[i]['start'], subtitle[i]['end'])
    
        #Saving the duration
        subtitle[i]['duration'] = subtitle[i]['clip'].duration
    
        #Generating the text clip
        subtitle[i]['text_clip'] = TextClip(subtitle[i]['text'], font='Ubuntu-Bold', fontsize=32, color='#423634', 
                                   size=(1000, 110), method='caption', bg_color='#b6b4b2', 
                                   align='center').set_opacity(0.75)
        #071a13 - Original colour
    
        #Adding the text clip to the clip
        subtitle[i]['clip'] = CompositeVideoClip([subtitle[i]['clip'], subtitle[i]['text_clip']
                                                  .set_pos(('center', 960))]).set_duration(subtitle[i]['duration'])
    
        #Adding a transition clip for the bit in between 2 subtitles
        subtitle[i]['transition'] = main_clip_logo.subclip(subtitle[i]['end'], subtitle[i+1]['start'])
        
        #Storing clips for concatenation
        clips_for_concatenation.append(subtitle[i]['clip'])
        clips_for_concatenation.append(subtitle[i]['transition'])
        
    #For the last clip, ensuring that it goes till the end
    else:
        
        #Extraction of clip
        subtitle[i]['clip'] = main_clip_logo.subclip(subtitle[i]['start'], main_clip_logo.duration)
        
        #Saving the duration
        subtitle[i]['duration'] = subtitle[i]['clip'].duration
    
        #Generating the text clip
        subtitle[i]['text_clip'] = TextClip(subtitle[i]['text'], font='Ubuntu-Bold', fontsize=32, color='#423634', 
                                   size=(1000, 110), method='caption', bg_color='#b6b4b2', 
                                   align='center').set_opacity(0.75)
        #071a13 - Original colour
        
        #Adding the text clip the clip
        subtitle[i]['clip'] = subtitle[i]['clip'] = CompositeVideoClip([subtitle[i]['clip'], subtitle[i]['text_clip']
                                                  .set_pos(('center', 960))]).set_duration(subtitle[i]['duration'])
    
        #Storing clip for concatenation
        clips_for_concatenation.append(subtitle[i]['clip'])

In [17]:
#Generating the main clip by concatenating all subclips
main_clip = concatenate_videoclips(clips_for_concatenation, method='chain')

In [18]:
#main_clip.ipython_display(width=640, maxduration=180)
#main_clip.write_videofile('/home/rohit/Videos/sasha_clips/test.mp4', logger=None)

### Adding subtitles using Subtitles Clip

subtitle background - #b6b4b2

subtitle font color - #071a13 
alternative font colors taken from InShot Pixel Color Picker - #394941, #32413e, #323d3a, #2c3531 

subtitle opacity - 80%

In [19]:
#generator = lambda txt: TextClip(txt, font='Ubuntu-Bold', fontsize=32, color='#071a13',
#                                 size=(1000, 110), method='caption', bg_color='#b6b4b2',
#                                 align='center').set_opacity(0.75)
#subtitles = SubtitlesClip("/home/rohit/Videos/sasha_clips/03_women_feedback.srt", generator)

In [20]:
#main_clip = CompositeVideoClip([main_clip_logo, subtitles.set_pos(('center', 960))])

In [21]:
#main_clip.ipython_display(width=640, maxduration=180)
#main_clip.ipython_display(width=640, t=25)

In [22]:
#main_clip.write_videofile('/home/rohit/Videos/sasha_clips/main_clip.mp4', logger=None)

### Post Clip Animation with Music Synced

We take a portion of the main clip after the clip, mute it, blur it and then add the end texts like "Getting Intimate with Sasha Cobra" etc and then the final logo.

In [23]:
#Defining how long we want the post clip animation to be
#This value comes from correctly cutting the music at the 
#beginning of the original file.
post_clip_duration = 9.51 #seconds

In [24]:
#Defining a function to generate timestamps for subclips
#Takes into account minute and seconds

def timestamp_gen(start, increment):
    """
    start - start time tuple (hours, min, seconds)
    increment - in seconds
    """
    
    #Setting stop tuple to be same as start
    stop = start
    
    #Converting to a list to increment as tuple is immutable
    temp_list = list(stop)
    
    #Adding length of post clip animation duration to the end of the main clip
    new_min = temp_list[2] + increment
    
    #Updating the list based on whether the incremented minute
    #is greater than or less than 60min as the hour has to be 
    #accounted for.
    if new_min < 60:
        temp_list[2] = new_min
    elif new_min > 60:
        temp_list[1] += 1
        temp_list[2] = new_min-60
        
    #Converting list back to tuple
    stop = tuple(temp_list)
    
    return stop

In [25]:
#Defining a function to blur a frame
def blur(image):
    
    """
    Returns a blurred image using gaussian filter
    with radius
    """
    
    return gaussian(image.astype(float), sigma=10)

In [26]:
#Computing time steps for the complete clip
post_clip_start = t_sub_end
post_clip_end = timestamp_gen(post_clip_start, post_clip_duration)

In [27]:
#Extracting the subclip
post_clip = VideoFileClip('/home/rohit/Videos/sasha_clips/gi_e1.mp4').subclip(post_clip_start, post_clip_end)

In [28]:
#Muting audio in post clip
post_clip_mute = post_clip.volumex(0)

In [29]:
#Cropping to instagram size
post_clip_crop = crop(post_clip_mute, width=h, height=h, x_center=0.5*w, y_center=0.5*h)

In [30]:
#Defining the subclip durations of the post clip
#We have 4 subclips based on the "eh" sound in the music

#Defining the timestamps in a dictionary
pc_t = {}

pc_t['1_start'] = (0, 0, 0.0)
pc_t['1_end'] = (0, 0, 1.111)
pc_t['2_start'] = pc_t['1_end']
pc_t['2_end'] = (0, 0, 3.544)
pc_t['3_start'] = pc_t['2_end']
pc_t['3_end'] = (0, 0, 6.040)
pc_t['4_start'] = pc_t['3_end']
pc_t['4_end'] = (0, 0, 8.47)
pc_t['5_start'] = pc_t['4_end']
pc_t['5_end'] = (0, 0, post_clip.duration)

In [31]:
#Extracting the subclips corresponding to the subclip stamps
#also applying blur after extraction
#storing in a dictionary
pc = {}

for i in range(1, 6):
    pc[i] = post_clip_crop.subclip(pc_t[str(i)+'_start'], pc_t[str(i)+'_end']) 
    pc[i] = pc[i].fl_image(blur)

In [32]:
#Defining text clips
getting_intimate_1 = (TextClip('GETTING INTIMATE', font='Montserrat-SemiBold', fontsize=85, color='white',
                            align='center', kerning=10)
                      .set_position('center', 'center')
                      .set_duration(pc[1].duration)
                      .crossfadein(0.3))

getting_intimate_2 = (TextClip('GETTING INTIMATE', font='Montserrat-SemiBold', fontsize=85, color='white',
                            align='center', kerning=10)
                      .set_position('center', 'center')
                      .set_duration(pc[2].duration)
                      .crossfadeout(0.3))

with_sc = (TextClip('WITH SASHA COBRA', font='Montserrat-SemiBold', fontsize=30, color='white', align='center')
                    .set_position((0.652, 0.55), relative=True)
                    .set_duration(pc[2].duration)
                    .crossfadein(0.3)
                    .crossfadeout(0.3))

watch_fv = (TextClip('Watch full video series', font='Montserrat-SemiBold', 
                    fontsize=40, color='white', align='center')
                    .set_position(('center', 'center'))
                    .set_duration(pc[3].duration)
                    .crossfadein(0.3)
                    .crossfadeout(0.3))

at_sc = (TextClip('at sashacobra.com', font='Montserrat-SemiBold',
                  fontsize=40, color='white', align='center')
                  .set_position(('center', 0.55), relative=True)
                  .set_duration(pc[3].duration)
                  .crossfadein(0.3)
                  .crossfadeout(0.3))

In [33]:
#Defining final logo image clip
final_logo_4 = (ImageClip("/home/rohit/Videos/sasha_clips/logos/logo_name/logo_name_21.png")
                .set_duration(pc[4].duration)
                .set_pos(("center", "center"))
                .resize(0.65)
                .crossfadein(0.3))

final_logo_5 = (ImageClip("/home/rohit/Videos/sasha_clips/logos/logo_name/logo_name_21.png")
                .set_duration(pc[5].duration)
                .set_pos(("center", "center"))
                .resize(0.65)
                .crossfadeout(pc[5].duration))

In [34]:
#Generating composite video clips
comp1 = CompositeVideoClip([pc[1], getting_intimate_1])
comp2 = CompositeVideoClip([pc[2], getting_intimate_2, with_sc])
comp3 = CompositeVideoClip([pc[3], watch_fv, at_sc])
comp4 = CompositeVideoClip([pc[4], final_logo_4])
comp5 = CompositeVideoClip([pc[5], final_logo_5])

post_clip_with_text = concatenate_videoclips([comp1, comp2, comp3, comp4, comp5])

In [35]:
#Setting post_clip audio

#Importing audio file
post_clip_audio = AudioFileClip('/home/rohit/Videos/sasha_clips/audio/intro_audio.wav')

#Setting audio
post_clip_final = post_clip_with_text.set_audio(post_clip_audio)

#Audio fade in and out effects
post_clip_final = post_clip_final.audio_fadein(0.3)
post_clip_final = post_clip_final.audio_fadeout(pc[5].duration)

In [36]:
#post_clip_final.write_videofile('/home/rohit/Videos/sasha_clips/post_clip_ani.mp4', logger=None)

In [37]:
final_export = concatenate_videoclips([main_clip, post_clip_final])
#final_export.ipython_display(width=640, maxduration=180)
final_export.write_videofile('/home/rohit/Videos/sasha_clips/09_love_programming.mp4', logger=None)

### Post Clip Animation without music sync
The goal here is to take next six seconds of the main video after the clip, blur it, mute it, put the text for Getting Intimate with Sasha Cobra and then watch the full video series on sashacobra.com and then the logo animation.  

In [38]:
##Defining how long we want the post clip animation to be
#post_clip_duration = 9.51 #seconds

##Defining number of subclips
#num_sub = 3

In [39]:
#Defining a function to generate timestamps for subclips
#Takes into account minute and seconds
#
#def timestamp_gen(start, increment):
#    """
#    start - start time tuple (hours, min, seconds)
#    increment - in seconds
#    """
#    
#    #Setting stop tuple to be same as start
#    stop = start
#    
#    #Converting to a list to increment as tuple is immutable
#    temp_list = list(stop)
#    
#    #Adding length of post clip animation duration to the end of the main clip
#    new_min = temp_list[2] + increment
#    
#    #Updating the list based on whether the incremented minute
#    #is greater than or less than 60min as the hour has to be 
#    #accounted for.
#    if new_min < 60:
#        temp_list[2] = new_min
#    elif new_min > 60:
#        temp_list[1] += 1
#        temp_list[2] = new_min-60
#        
#    #Converting list back to tuple
#    stop = tuple(temp_list)
#    
#    return stop

In [40]:
##Defining a function to blur a frame
#def blur(image):
#    
#    """
#    Returns a blurred image using gaussian filter
#    with radius 2
#    """
#    
#    return gaussian(image.astype(float), sigma=7)

In [41]:
##Computing time steps for the complete clip
#post_clip_start = t_sub_end
#post_clip_end = timestamp_gen(post_clip_start, post_clip_duration)

In [42]:
##Extracting the subclip
#post_clip = VideoFileClip('/home/rohit/Videos/sasha_clips/gi_e2.mp4').subclip(post_clip_start, post_clip_end)

In [43]:
##Muting audio in post clip
#post_clip_mute = post_clip.volumex(0)

In [44]:
##Cropping to instagram size
#crop_clip = crop(post_clip_mute, width=h, height=h, x_center=0.5*w, y_center=0.5*h)

In [45]:
#Computing timestamps for the 3 subclips
#pc_t = {}
#pc_t['1_start'] = (0,0,0.0)
#
#sub_increment = post_clip_duration/num_sub 
#
#for i in range(1, num_sub+1):
#    
#    #Computing and saving the end timestamp
#    pc_t[str(i)+'_end'] = timestamp_gen(pc_t[str(i)+'_start'], sub_increment)
#    
#    #Setting the beginning the next subclip as the end of first one
#    if i <=3:
#        pc_t[str(i+1)+'_start'] = pc_t[str(i)+'_end']

In [46]:
#Creating subclips from post-clip and applying blur
#pc = {}
#
#for i in range(1, num_sub+1):
#    print('generating subclip {}'.format(i))
#    pc[i] = crop_clip.subclip(pc_t[str(i)+'_start'], pc_t[str(i)+'_end'])
#    pc[i] = pc[i].fl_image(blur)

In [47]:
##Adding logo
#sub_clip_logo = (ImageClip("/home/rohit/Videos/sasha_clips/logos/logo_symbol_only/symbol3.png")
#                 .set_duration(pc[1].duration)
#                 .set_pos((-35, -40))
#                 .resize(0.25))
#
#sub_clip_handle = (TextClip('@sashacobra', font='Ubuntu-SemiBold', fontsize=28, color='white', align='center')
#                 .set_position((125, 57))
#                 .set_duration(pc[1].duration))
#
#for i in range(1, num_sub):
#    if i==2:
#        pc[i] = CompositeVideoClip([pc[i], sub_clip_logo.crossfadeout(0.1*pc[i].duration), 
#                                   sub_clip_handle.crossfadeout(0.1*pc[i].duration)])
#    else:
#        pc[i] = CompositeVideoClip([pc[i], sub_clip_logo, sub_clip_handle])

In [48]:
##Defining text clips
#getting_intimate = (TextClip('GETTING INTIMATE', font='Montserrat-SemiBold', fontsize=85, color='white',
#                            align='center', kerning=10)
#                    .set_position('center', 'center')
#                    .set_duration(pc[1].duration)
#                    .crossfadein(0.3*pc[1].duration)
#                    .crossfadeout(0.1*pc[1].duration))
#
#with_sc = (TextClip('WITH SASHA COBRA', font='Montserrat-SemiBold', fontsize=30, color='white', align='center')
#                    .set_position((0.652, 0.55), relative=True)
#                    .set_start(0.4*pc[1].duration)
#                    .set_duration(0.6*pc[1].duration)
#                    .crossfadein(0.3*pc[1].duration)
#                    .crossfadeout(0.1*pc[1].duration))
#
#watch_fv = (TextClip('Watch full video series', font='Montserrat-SemiBold', 
#                    fontsize=40, color='white', align='center')
#                    .set_position(('center', 'center'))
#                    .set_duration(pc[2].duration)
#                    .crossfadein(0.3*pc[2].duration)
#                    .crossfadeout(0.1*pc[2].duration))
#
#at_sc = (TextClip('at sashacobra.com', font='Montserrat-SemiBold',
#                  fontsize=40, color='white', align='center')
#                  .set_position(('center', 0.6), relative=True)
#                  .set_duration(pc[2].duration)
#                  .crossfadein(0.3*pc[2].duration)
#                  .crossfadeout(0.1*pc[2].duration))

In [49]:
##Defining final logo image clip
#final_logo = (ImageClip("/home/rohit/Videos/sasha_clips/logos/logo_name/logo_name_21.png")
#              .set_duration(pc[3].duration)
#              .set_pos(("center", "center"))
#              .resize(0.65)
#              .crossfadein(0.3*pc[3].duration)
#              .crossfadeout(0.1*pc[3].duration)) 

In [50]:
##Adding text to first subclip
#test1 = CompositeVideoClip([pc[1], getting_intimate, with_sc])
#test2 = CompositeVideoClip([pc[2], watch_fv, at_sc])
#test3 = CompositeVideoClip([pc[3], final_logo])
#test = concatenate_videoclips([test1, test2, test3])
#
#test2.ipython_display(width=640, t=1, verbose=False)
##test2.save_frame('/home/rohit/Downloads/wfv_after.png', t=1)

In [51]:
##Setting the audio of the post clip animation from the first part of original video
##And fading the audio out in last few moments
#intro_clip = VideoFileClip(video_file_path).subclip((0,0,0.0), (0,0,post_clip_duration))
#post_ani_final_clip = test.set_audio(intro_clip.audio)
#post_ani_final_clip = post_ani_final_clip.audio_fadein(0.3*post_ani_final_clip.duration)
#post_ani_final_clip = post_ani_final_clip.audio_fadeout(0.3*post_ani_final_clip.duration)

In [52]:
#Writing audio_file
#intro_audio = intro_clip.audio
#intro_audio.write_audiofile("/home/rohit/Videos/sasha_clips/intro_audio.wav", 44100, 2, 2000,"pcm_s32le")

In [53]:
#post_ani_final_clip.write_videofile('/home/rohit/Videos/sasha_clips/post_clip_ani.mp4', logger=None)

In [54]:
#final_export = concatenate_videoclips([main_clip, post_ani_final_clip])
#final_export.write_videofile('/home/rohit/Videos/sasha_clips/men_purge.mp4', logger=None)